# ECE324 

<img src = "https://geekflare.com/wp-content/uploads/2021/12/ai-projects-for-beginners.png" alt="image" />

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow
!pip install datasets
!pip install image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/cola

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.transforms import Compose, Resize, ToTensor, ToPILImage, RandomHorizontalFlip, RandomRotation, RandomResizedCrop, ColorJitter, ToTensor

import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image
import json
import urllib.request
from io import BytesIO
import pandas as pd
import uuid

import nltk
nltk.download("punkt")

from nltk import tokenize

from datasets import load_dataset
from pathlib import Path

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

datasetImageList = []
datasetLabelList = []

stable_diffusion_dataset = load_dataset('poloclub/diffusiondb', 'large_first_5k')["train"]
dalle2_dataset = datasets.ImageFolder('drive/MyDrive/diffusion_data/images/')


with open('drive/MyDrive/diffusion_data/midjourney/general-01_2022_06_20.json', 'r') as f:
  data = json.load(f)
  data = data["messages"]
  print(data)
  counter = 0
  for image in data:
    if (counter > 5000):
      break
    if (len(image) > 0 and len(image[0]["attachments"]) > 0):
      req = urllib.request.Request(url=image[0]["attachments"][0]["proxy_url"], headers=headers)
      try:
        with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"images/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          datasetImageList.append(pil_image)
          datasetLabelList.append(torch.tensor(3))
          counter = counter + 1
      except Exception as e:
        print(e)
        pass

  0%|          | 0/1 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:


df = pd.read_parquet('drive/MyDrive/diffusion_data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet')

for index, row in df.iterrows():
    if (index > 5000):
      break
    req = urllib.request.Request(url=row["URL"], headers=headers)
    try:
      with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"images/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          datasetImageList.append(file_path)
          datasetLabelList.append(torch.tensor(0))
    except: 
      pass


del df

In [ ]:
counter = 0

for data in dalle2_dataset:
  image = np.array(data[0])[:3]
  label = torch.tensor(2)
  datasetImageList.append(image)
  datasetLabelList.append(label)
  counter += 1

for data in stable_diffusion_dataset:
  image = np.array(data['image'])[:3]
  label = torch.tensor(1)
  datasetImageList.append(image)
  datasetLabelList.append(label)

del dalle2_dataset
del stable_diffusion_dataset

bruh
bruh
bruh
bruh


In [ ]:
def resize_and_convert(image, size=(254, 254)):
    if not isinstance(image, np.ndarray):
        image = np.array(image)
    pil_image = Image.fromarray(image).convert('RGB')
    pil_image = pil_image.resize(size)
    return np.transpose(np.array(pil_image), (2, 0, 1))

augmentations = Compose([
    RandomHorizontalFlip(p=0.5),
    RandomRotation(degrees=15),
    RandomResizedCrop(size=(254, 254), scale=(0.8, 1.0), ratio=(0.75, 1.333)),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    ToTensor()
])


class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_list, label_list, transform):
        self.image_list = image_list
        self.label_list = label_list
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image = self.image_list[idx]
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        elif isinstance(image, np.ndarray):
            image = Image.fromarray(image).convert("RGB")

        image = image.resize((254, 254))
        label = self.label_list[idx]
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
batch_size = 64
dataset = ImageDataset(datasetImageList, datasetLabelList, augmentations)

train_size = int(len(dataset) * 0.90)
val_size = len(dataset) - train_size
train_indices = torch.randperm(len(dataset))[:train_size]
val_indices = torch.arange(train_size, len(dataset))

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)



device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Loading the previous model if it exists and then training the model based on data. Save the model after each epoch.

In [ ]:
model = torch.load("drive/MyDrive/diffusion_data/mode.pt")

num_features = model.fc.in_features
#model.fc = nn.Linear(num_features, 4)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.000001)

for param in model.parameters():
  param.requires_grad = True
num_epochs = 20
for epoch in range(num_epochs):



    # Forward pass
    model.train()
    for inputs, labels in train_dataloader:
      # Zero the gradients
      optimizer.zero_grad()

      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())

      # Compute the loss
      labels = labels.to(device)
      loss = criterion(outputs, labels)

      # Backward pass
      loss.backward()
      optimizer.step()
    print(epoch)
    torch.save(model, "drive/MyDrive/diffusion_data/mode.pt")

    model.eval()
    total_correct = 0
    total_samples = 0
    total_loss = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
          # Forward pass
          inputs = inputs.to(device)
          outputs = model(inputs.float())
          _, predicted = torch.max(outputs.data, 1)
          
          labels = labels.to(device)
          loss = criterion(outputs, labels)
          total_loss += loss.item()

          total_samples += labels.size(0)
          total_correct += (predicted == labels).sum().item()

    accuracy = total_correct / total_samples
    print('Validation accuracy:', accuracy)
    print("Loss", total_loss)


0
Validation accuracy: 0.9156378600823045
Loss 3.766630917787552
1
Validation accuracy: 0.8724279835390947
Loss 4.033371031284332
2
Validation accuracy: 0.8724279835390947
Loss 4.010570198297501
3
Validation accuracy: 0.8353909465020576
Loss 4.006787955760956
4
Validation accuracy: 0.8744855967078189
Loss 3.9909191727638245


KeyboardInterrupt: ignored

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

datasetImageList = []
datasetLabelList = []

stable_diffusion_dataset = load_dataset('poloclub/diffusiondb', 'large_random_1k')["train"]
dalle2_dataset = datasets.ImageFolder('drive/MyDrive/diffusion_data/images/')

counter = 0


dalle2ImageList = []
dalle2LabelList = []
for data in dalle2_dataset:
  if (counter > 100):
    break
  image = np.array(data[0])[:3]
  label = torch.tensor(2)
  dalle2ImageList.append(image)
  dalle2LabelList.append(label)
  counter += 1



stableDiffusionImageList = []
stableDiffusionLabelList = []
counter = 0
for data in stable_diffusion_dataset:
  if (counter > 100):
    break
  image = np.array(data['image'])[:3]
  label = torch.tensor(1)
  stableDiffusionImageList.append(image)
  stableDiffusionLabelList.append(label)
  counter += 1

del dalle2_dataset
del stable_diffusion_dataset



midjourneyImageList = []
midjourneyLabelList = []
with open('/content/drive/MyDrive/diffusion_data/midjourney/general-01_2022_07_04.json', 'r') as f:
  data = json.load(f)
  data = data["messages"]
  counter = 0
  for image in data:
    if (counter > 100):
      break
    if (len(image) > 0 and len(image[0]["attachments"]) > 0):
      req = urllib.request.Request(url=image[0]["attachments"][0]["proxy_url"], headers=headers)
      try:
        with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"mid/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          midjourneyImageList.append(pil_image)
          midjourneyLabelList.append(torch.tensor(3))
          counter = counter + 1
      except Exception as e:
        print(e)
        pass


realImageList = []
realLabelList = []
df = pd.read_parquet('/content/drive/MyDrive/diffusion_data/part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet')

for index, row in df.iterrows():
    if (index > 100):
      break
    req = urllib.request.Request(url=row["URL"], headers=headers)
    try:
      with urllib.request.urlopen(req, timeout=10) as url:
          image_data = url.read()
          pil_image = Image.open(BytesIO(image_data))

          file_path = f"real/{uuid.uuid4().hex}.jpg"

          pil_image.save(file_path)

          realImageList.append(file_path)
          realLabelList.append(torch.tensor(0))
    except: 
      pass


del df

Generating train split: 0 examples [00:00, ? examples/s]

Dataset diffusiondb downloaded and prepared to /root/.cache/huggingface/datasets/poloclub___diffusiondb/large_random_1k/0.9.1/547894e3a57aa647ead68c9faf148324098f47f2bc1ab6705d670721de9d89d1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
augmentations = Compose([
    ToTensor()
])
batch_size = 64
realDataset = ImageDataset(realImageList, realLabelList, transform=augmentations)
midjourneyDataset = ImageDataset(midjourneyImageList, midjourneyLabelList, transform=augmentations)
stableDiffusionDataset = ImageDataset(stableDiffusionImageList, stableDiffusionLabelList, transform=augmentations)
dalle2Dataset = ImageDataset(dalle2ImageList, dalle2LabelList, transform=augmentations)


realDataloader = DataLoader(realDataset, batch_size=batch_size)
midjourneyDataloader = DataLoader(midjourneyDataset, batch_size=batch_size)
stableDiffusionDataloader = DataLoader(stableDiffusionDataset, batch_size=batch_size)
dalle2Dataloader = DataLoader(dalle2Dataset, batch_size=batch_size)

In [ ]:
model = torch.load("drive/MyDrive/diffusion_data/mode.pt")


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()

model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in realDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Real:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")

model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in midjourneyDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Midjourney:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")
model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in stableDiffusionDataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Stable Diffusion:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")
model.eval()
total_correct = 0
total_samples = 0
total_loss = 0
with torch.no_grad():
    for inputs, labels in dalle2Dataloader:
      # Forward pass
      inputs = inputs.to(device)
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      
      labels = labels.to(device)
      loss = criterion(outputs, labels)
      total_loss += loss.item()

      total_samples += labels.size(0)
      total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print("Dalle 2:")
print('Validation accuracy:', accuracy)
print("Loss", total_loss)
print("")






Real:
Validation accuracy: 0.6585365853658537
Loss 1.578683078289032

Midjourney:
Validation accuracy: 0.801980198019802
Loss 1.15653795003891

Stable Diffusion:
Validation accuracy: 0.45544554455445546
Loss 2.891270160675049

Dalle 2:
Validation accuracy: 0.19801980198019803
Loss 3.552420973777771

